# Introduction
This notebook demonstrates the integration capabilities of roboquant with OANDA. OANDA is a broker with a focus on FOREX, but also offers CFD and several commodities. 

<img src="https://www.oanda.com/assets/images/oanda-logo.20eebbd07880.svg" alt="Binance" width="400"/>



There are currently three types of integration available:

1. Back testing with historic data from OANDA 
2. Live testing with live data from OANDA 
3. Paper trading on OANDA (WIP)


By default the required OANDA classes are not imported, so we need to do that first. This will load the Feed and Broker classes for integrating with OANDA.

In [ ]:
%use roboquant
import org.roboquant.oanda.*

// By default USD, EUR and GBP are displayed with 2 decimals, but for FOREX trading we want more precision in order to see small differences
// So we increase it by 3
Currency.increaseDigits(3)

Welcome()

# Back Testing
The first example shows how to use historic data from OANDA in a back test. To use Alpaca you'll need to authenticate with a Key and Secret. There are several ways to pass them to the feed, either directly or through system environment or a property file:


<div class="alert alert-block alert-info">
Make sure you use the demo account since roboquant is still very much in Beta
</div>

In [ ]:
val feed = OANDAHistoricFeed()
val timeFrame = TimeFrame.parse("2021-08-10", "2021-08-11")
feed.retrieveCandles("EUR_USD", timeFrame = timeFrame)
feed.assets.summary()

In [ ]:
PriceBarChart(feed, feed.assets.first())

We'll define strategy of which the buy and sell signals are based on candlestick patterns. Although trading Forex is like any another asset class, there are some configuration parameters that deviate from assets classes (less spread, more shorting)
OANDA.roboquant presets these parameters

In [ ]:
val strategy = TAStrategy(15)

// We want to generate a BUY signal if we detect a Three White Soldiers pattern
strategy.buy { ta.cdl3WhiteSoldiers(it) }

// We want to generate a SELL signal if we detect the candlestick pattern Three Black Crows 
strategy.sell { ta.cdl3BlackCrows(it) }


val roboquant = OANDA.roboquant(strategy, AccountSummary())

In [ ]:
roboquant.run(feed)

## Results

In [ ]:
val account = roboquant.broker.account
account.fullSummary()

In [ ]:
val metric = roboquant.logger.getMetric("account.value")
MetricChart(metric)

In [ ]:
feed.assets.forEach { 
    PriceBarChart(feed, it, account.trades).render()
}

# Live Testing
Now we create an instance of the OANDALiveFeed and subscribe to three forex pairs. In order to keep it simple, in this example we subscribe to pairs denoted in USD so we can trade without having to define a currency converter. 
If you run this cell will FOREX exchanges are closed, there will no orders and trades of course.

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook("GBP_USD", "EUR_USD", "AUD_USD")

In [ ]:
val strategy = EMACrossover()
val priceRecorder = PriceRecorder(*feed.assets.toTypedArray())
val roboquant = OANDA.roboquant(strategy, priceRecorder, AccountSummary())

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. We'll run it for 5 minutes, but you can change this. 

In [ ]:
val timeFrame = TimeFrame.next(5.minutes)
roboquant.run(feed, timeFrame)

## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
val account = roboquant.broker.account
account.fullSummary()

In [ ]:
for (asset in priceRecorder.assets) PriceChart(priceRecorder, asset, account.trades).render()

In [ ]:
val logger = roboquant.logger
logger.summary().print()

In [ ]:
logger.metricNames.summary()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChartByAsset(account.trades)

# Paper Trading
And just as it was the case in the previous section, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated.

<div class="alert alert-block alert-warning">
The integration with the trading API is still very much work in progress, so only use this with a paper trading account
</div>

In [ ]:
val broker = OANDABroker(enableOrders = true)
val account = broker.account

In [ ]:
account.fullSummary()

## Run
Get the data we want to use to feed our strategy. The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. This time we'll run it for 5 minutes.

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook("GBP_USD", "EUR_USD", "EUR_GBP")

In [ ]:
val strategy = EMACrossover(7, 13) // Use EMA Crossover strategy
val policy = DefaultPolicy(shorting = true) // We want to short if we do Forex trading
val assets = feed.assets.toTypedArray()
val priceRecorder = PriceRecorder(*assets)
val roboquant = Roboquant(strategy, AccountSummary(), priceRecorder, policy = policy, broker = broker)

In [ ]:
val timeFrame = TimeFrame.next(10.minutes) // restrict the time time from now for the next minutes
roboquant.run(feed, timeFrame)

## Results

In [ ]:
account.fullSummary()

In [ ]:
for (asset in assets) PriceChart(priceRecorder, asset, account.trades).render()

In [ ]:
val metricData = roboquant.logger.getMetric("account.pnl")
MetricChart(metricData)